# 🧠 Reddit User Persona Generator (Jupyter Notebook)

In [ ]:
!pip install praw openai

In [ ]:
import praw
import openai
import re
from datetime import datetime

# 🔐 Replace with your credentials
REDDIT_CLIENT_ID = "lyVBX1jZLAKF5Rp-5RHfA"
REDDIT_CLIENT_SECRET = "8cBtiBvAlP2gw5lAGhf10dnuKNXzfw"
REDDIT_USER_AGENT = "user-persona-script by u/User_mohammadi"
OPENAI_API_KEY = "your_openai_api_key"

openai.api_key = OPENAI_API_KEY


In [ ]:
def extract_username(url):
    match = re.search(r"reddit\.com/user/([A-Za-z0-9_-]+)", url)
    return match.group(1) if match else None


In [ ]:
def scrape_user_data(username):
    reddit = praw.Reddit(
        client_id=REDDIT_CLIENT_ID,
        client_secret=REDDIT_CLIENT_SECRET,
        user_agent=REDDIT_USER_AGENT
    )
    user = reddit.redditor(username)
    comments, posts = [], []

    for comment in user.comments.new(limit=50):
        comments.append(f"Comment: {comment.body[:500]}")
    for post in user.submissions.new(limit=20):
        content = post.selftext if post.selftext else "(No text content)"
        posts.append(f"Post: {post.title}\n{content[:500]}")
    return comments, posts


In [ ]:
def generate_persona_from_text(text):
    prompt = f"""
You are an AI persona analyst. Based on the following Reddit posts and comments, generate a detailed user persona with:

- Username
- Age range (guess)
- Interests & hobbies
- Personality traits
- Writing style
- Any political/cultural leaning
- Frequently discussed topics

Under each trait, cite a related comment or post snippet.

Text:
{text}
"""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message['content']


In [ ]:
def save_to_file(persona_text, username):
    filename = f"user_persona_{username}_{datetime.now().strftime('%Y%m%d_%H%M')}.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(persona_text)
    print(f"✅ Saved to: {filename}")


In [ ]:
reddit_url = input("Enter Reddit user profile URL: ").strip()
username = extract_username(reddit_url)

if not username:
    print("❌ Invalid Reddit URL")
else:
    print(f"🔍 Scraping data for u/{username}...")
    comments, posts = scrape_user_data(username)
    combined_text = "\n\n".join(comments + posts)

    print("🤖 Generating persona...")
    persona = generate_persona_from_text(combined_text)

    save_to_file(persona, username)
